In [1]:
# This notebook is a testing ground for building OpenAI Agents. 

In [2]:
# Jupyter notebooks spin up an asyncio loop, so the below code makes sure OpenAI's run_sync function works. 
import nest_asyncio
nest_asyncio.apply()

In [3]:
import json
from agents import Agent, Runner, GuardrailFunctionOutput
from prompts import (KLIRO_PROMPT, INHERITANCE_RISK_AGENT_PROMPT, EXECUTOR_RISK_AGENT_PROMPT)
from guardrails import PIIOutput, PII_GUARDRAIL_AGENT, pii_guardrail

In [4]:
# Create the specialist agents first so that the main Kliro agent can "see" these agents in its handoffs
inheritance_risk_agent = Agent(
    name="Inheritance Risk Agent",
    handoff_description="Calculates the total estate value and breakdown of costs at risk",
    instructions=INHERITANCE_RISK_AGENT_PROMPT
)

executor_risk_agent = Agent(
    name="Executor Risk Agent",
    handoff_description="Estimates time, effort, and risk level for estate settlement",
    instructions=EXECUTOR_RISK_AGENT_PROMPT
)

In [5]:
# Then create our main Kliro Agent that will speak with the user and delegate to our specialist agents
kliro_agent = Agent(
    name="Kliro Agent", 
    instructions=KLIRO_PROMPT,
    handoff_description="Master orchestrator guiding the inheritance conversation",
    handoffs=[
      inheritance_risk_agent, 
      executor_risk_agent,
      # later: action_plan_agent, estate_map_agent, etc.
    ], 
    input_guardrails=[pii_guardrail]
)

In [8]:
async def main():
    # ——— Test 1: simple greeting ———
    greet = await Runner.run(kliro_agent, "Hi Kliro!")
    print("=== Greeting ===")
    print(greet.final_output, "\n")

    # ——— Test 2: trigger Inheritance Risk Agent ———
    assets = [
        {"name": "Home",        "type": "real_estate", "value": 500000},
        {"name": "Investments", "type": "securities",  "value": 200000}
    ]
    prompt_risk = (
        "Here are my parents' assets: " + json.dumps(assets) +
        ". Please calculate the inheritance risk."
    )
    risk = await Runner.run(kliro_agent, prompt_risk)
    print("=== Inheritance Risk ===")
    print(risk.final_output, "\n")

    # ——— Test 3: trigger Executor Risk Agent ———
    executor_info = {
        "total_value": 700000,
        "executor_experience": "medium",
        "estate_complexity": "high",
        "family_dynamics": "contentious"
    }
    prompt_exec = (
        "Given this executor context: " + json.dumps(executor_info) +
        ", please estimate the settlement timeline and executor risk."
    )
    exec_res = await Runner.run(kliro_agent, prompt_exec)
    print("=== Executor Risk ===")
    print(exec_res.final_output)


    # ——— Test 4: trigger PII guardrail ———
    pii_input = "Here is my SSN: 123-45-6789 and my credit card 4111 1111 1111 1111"
    guard_res = await Runner.run(kliro_agent, pii_input)
    print("=== PII Guardrail ===")
    # guard_res is an InputGuardrailResult, so the parsed output is in .output_info
    print(guard_res.output_info.json(), "\n")

In [9]:
await main()

=== Greeting ===
Hi! I'm Kliro, your AI assistant to help you manage your inheritance. Let's start off with a few basic questions.

- Can you tell me about your family dynamic? Where do your parents live? Are they still together, or are they divorced? 
- Where about do you live? 
- Do you have any siblings? Please share any information about them that will give me a comprehensive overview of your family dynamic!

If your parents are divorced, note that I can only focus on one estate at a time, so let me know which one you want to concentrate on. 

=== Inheritance Risk ===
{
    "total_value": 700000,
    "breakdown": {
        "probate_cost": 10500.0,
        "capital_gains_tax": 175000.0,
        "other_fees": 1000
    },
    "value_at_risk": 186500.0
}

Your estate is $700,000. Probate fees $10,500; capital gains tax $175,000; other fees $1,000—total $186,500 at risk. 

=== Executor Risk ===
```json
{
  "settlement_time_months": 12,
  "effort_hours": 120,
  "executor_risk": "high"
}


InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [ ]:
result = Runner.run_sync(kliro_agent, "Hi Kliro.")
print(result.final_output)

In [10]:
import gradio as gr

async def kliro_chat(user_message, chat_history=[]):
    # call the agent
    result = await Runner.run(kliro_agent, user_message)
    reply = result.final_output
    chat_history = chat_history + [(user_message, reply)]
    return chat_history, chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    state   = gr.State([])
    txt     = gr.Textbox(placeholder="Type a message and hit enter")

    txt.submit(
        kliro_chat, 
        inputs=[txt, state], 
        outputs=[chatbot, state]
    )

demo.launch()

/var/folders/yc/yk794q4d3qj7f5jdv1w0l2tm0000gn/T/ipykernel_20610/525858268.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
